## Import packages

In [1]:
import numpy as np 
import csv 
import glob
import pandas as pd
import os
import sys
from tqdm import tqdm

# Get the path to the parent directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

# Import utils folder
from utils.twostep_support import *    
from models import *

## Set number of parameter sets (number of synthetic participants) and select model class

In [2]:
model = "RL_ddm_biased" # RL, RL_ddm, RL_ddm_biased, AI, or AI_ddm
mtype = 3 # 0, 1, 2 or 3 (only relevant if model = AI or AI_ddm)
drmtype = "linear" # Drift rate model: linear, sigmoid, sigmoid_single_v_mod, sigmoid_single_v_max
dataset = 'magic_carpet_2020'

In [3]:
#Number of parameter sets or 'synthetic participants'
if dataset == 'magic_carpet_2020':
    n_param_sets = 23 
elif dataset == 'magic_carpet_2023':
    n_param_sets = 46 

# Learning algorithm
if model in ("RL", "RL_ddm", "RL_ddm_biased"):
    learning = "RL"
elif model in ("AI", "AI_ddm"):
    learning = "PSM"

## Generate parameter sets (synthetic participants)

In [4]:
# Different model setups
if model == "RL":
    p_names = ["Synthetic_lr1", 
               "Synthetic_lr2", 
               "Synthetic_lam", 
               "Synthetic_b1", 
               "Synthetic_b2", 
               "Synthetic_p", 
               "Synthetic_w"]
    lower_bounds = np.array([0, 0, 0, 0, 0, -1, 0])
    upper_bounds = np.array([1, 1, 1, 20, 20, 1, 1])

elif model == "RL_ddm":
    if drmtype == "linear":
        p_names = ["Synthetic_lr1", "Synthetic_lr2", "Synthetic_lam","Synthetic_w","Synthetic_p" , "Synthetic_a_bs", "Synthetic_ndt", "Synthetic_v_stage_0", "Synthetic_v_stage_1"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 10, 10])
    elif drmtype == "sigmoid_single_v_mod":
        p_names = ["Synthetic_lr1", "Synthetic_lr2", "Synthetic_lam", "Synthetic_w","Synthetic_p","Synthetic_a_bs", "Synthetic_ndt", "Synthetic_v_max_stage_0", "Synthetic_v_max_stage_1", "Synthetic_v_mod"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 10, 10, 1])
    elif drmtype == "sigmoid":
        p_names = ["Synthetic_lr1", "Synthetic_lr2", "Synthetic_lam", "Synthetic_w","Synthetic_p","Synthetic_a_bs", "Synthetic_ndt", "Synthetic_v_max_stage_0", "Synthetic_v_max_stage_1", "Synthetic_v_mod_stage_0", "Synthetic_v_mod_stage_1"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 10, 10, 1, 1])
    elif drmtype == "sigmoid_single_v_max":
        p_names = ["Synthetic_lr1", "Synthetic_lr2", "Synthetic_lam", "Synthetic_w","Synthetic_p","Synthetic_a_bs", "Synthetic_ndt", "Synthetic_v_max", "Synthetic_v_mod_stage_0", "Synthetic_v_mod_stage_1"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 10, 1, 1])
elif model == "RL_ddm_biased":
    if drmtype == "linear":
        p_names = ["Synthetic_lr1", "Synthetic_lr2", "Synthetic_lam","Synthetic_w","Synthetic_p" , "Synthetic_a_bs", "Synthetic_ndt", "Synthetic_z_prime", "Synthetic_v_stage_0", "Synthetic_v_stage_1"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 0.5, 10, 10])
    elif drmtype == "sigmoid_single_v_mod":
        p_names = ["Synthetic_lr1", "Synthetic_lr2", "Synthetic_lam", "Synthetic_w","Synthetic_p","Synthetic_a_bs", "Synthetic_ndt", "Synthetic_z_prime", "Synthetic_v_max_stage_0", "Synthetic_v_max_stage_1", "Synthetic_v_mod"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 0.5, 10, 10, 1])
    elif drmtype == "sigmoid":
        p_names = ["Synthetic_lr1", "Synthetic_lr2", "Synthetic_lam", "Synthetic_w","Synthetic_p","Synthetic_a_bs", "Synthetic_ndt", "Synthetic_z_prime", "Synthetic_v_max_stage_0", "Synthetic_v_max_stage_1", "Synthetic_v_mod_stage_0", "Synthetic_v_mod_stage_1"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 0.5, 10, 10, 1, 1])
    elif drmtype == "sigmoid_single_v_max":
        p_names = ["Synthetic_lr1", "Synthetic_lr2", "Synthetic_lam", "Synthetic_w","Synthetic_p","Synthetic_a_bs", "Synthetic_ndt", "Synthetic_z_prime", "Synthetic_v_max", "Synthetic_v_mod_stage_0", "Synthetic_v_mod_stage_1"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 0.5, 10, 1, 1])

elif model == "AI":
    if mtype == 0:
        p_names = ["Synthetic_lr",
                   "Synthetic_vunsamp", 
                   "Synthetic_vsamp", 
                   "Synthetic_vps", 
                   "Synthetic_gamma1", 
                   "Synthetic_gamma2", 
                   "Synthetic_lam", 
                   "Synthetic_kappa_a", 
                   "Synthetic_prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 0.9, 20, 20, 10, 5, 0.8])
    elif mtype == 1:
        p_names = ["Synthetic_lr",
                   "Synthetic_vunsamp", 
                   "Synthetic_vps", 
                   "Synthetic_gamma1", 
                   "Synthetic_gamma2", 
                   "Synthetic_lam", 
                   "Synthetic_kappa_a", 
                   "Synthetic_prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 20, 20, 10, 5, 0.8])
    elif mtype == 2:
        p_names = ["Synthetic_lr", 
                   "Synthetic_vsamp", 
                   "Synthetic_vps", 
                   "Synthetic_gamma1", 
                   "Synthetic_gamma2", 
                   "Synthetic_lam", 
                   "Synthetic_kappa_a", 
                   "Synthetic_prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 20, 20, 10, 5, 0.8])
    elif mtype == 3:
        p_names = ["Synthetic_lr",
                   "Synthetic_vunsamp", 
                   "Synthetic_vsamp", 
                   "Synthetic_gamma1", 
                   "Synthetic_gamma2", 
                   "Synthetic_lam", 
                   "Synthetic_kappa_a", 
                   "Synthetic_prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 20, 20, 10, 5, 0.8]) 

elif model == "AI_ddm":
    if mtype == 0:
        if drmtype == "linear":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vsamp", 
                       "Synthetic_vps", 
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_stage_0", 
                       "Synthetic_v_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10])
        if drmtype == "sigmoid":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vsamp", 
                       "Synthetic_vps", 
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max_stage_0", 
                       "Synthetic_v_max_stage_1", 
                       "Synthetic_v_mod_stage_0", 
                       "Synthetic_v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1, 1])    
        if drmtype == "sigmoid_single_v_mod":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vsamp", 
                       "Synthetic_vps", 
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max_stage_0", 
                       "Synthetic_v_max_stage_1", 
                       "Synthetic_v_mod"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1]) 
        if drmtype == "sigmoid_single_v_max":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vsamp", 
                       "Synthetic_vps", 
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max", 
                       "Synthetic_v_mod_stage_0", 
                       "Synthetic_v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 1, 1])    
    elif mtype == 1:
        if drmtype == "linear":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vps",
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_stage_0", 
                       "Synthetic_v_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10])
        if drmtype == "sigmoid":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vps",
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max_stage_0", 
                       "Synthetic_v_max_stage_1", 
                       "Synthetic_v_mod_stage_0", 
                       "Synthetic_v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1, 1])
        if drmtype == "sigmoid_single_v_mod":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vps",
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max_stage_0", 
                       "Synthetic_v_max_stage_1", 
                       "Synthetic_v_mod"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1])
        if drmtype == "sigmoid_single_v_max":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vps",
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max", 
                       "Synthetic_v_mod_stage_0", 
                       "Synthetic_v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 1, 1])
    elif mtype == 2:
        if drmtype == "linear":
            p_names = ["Synthetic_lr", 
                       "Synthetic_vsamp", 
                       "Synthetic_vps",
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_stage_0", 
                       "Synthetic_v_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10])

        if drmtype == "sigmoid":
            p_names = ["Synthetic_lr", 
                       "Synthetic_vsamp", 
                       "Synthetic_vps",
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max_stage_0", 
                       "Synthetic_v_max_stage_1", 
                       "Synthetic_v_mod_stage_0", 
                       "Synthetic_v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1, 1])
        if drmtype == "sigmoid_single_v_mod":
            p_names = ["Synthetic_lr", 
                       "Synthetic_vsamp", 
                       "Synthetic_vps",
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max_stage_0", 
                       "Synthetic_v_max_stage_1", 
                       "Synthetic_v_mod"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1])
        if drmtype == "sigmoid_single_v_max":
            p_names = ["Synthetic_lr", 
                       "Synthetic_vsamp", 
                       "Synthetic_vps",
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max", 
                       "Synthetic_v_mod_stage_0", 
                       "Synthetic_v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 1, 1])
    elif mtype == 3:
        if drmtype == "linear":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vsamp", 
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_stage_0", 
                       "Synthetic_v_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10])

        if drmtype == "sigmoid":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vsamp", 
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max_stage_0", 
                       "Synthetic_v_max_stage_1", 
                       "Synthetic_v_mod_stage_0", 
                       "Synthetic_v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1, 1])
        
        if drmtype == "sigmoid_single_v_mod":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vsamp", 
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max_stage_0", 
                       "Synthetic_v_max_stage_1", 
                       "Synthetic_v_mod"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1])
        if drmtype == "sigmoid_single_v_max":
            p_names = ["Synthetic_lr",
                       "Synthetic_vunsamp", 
                       "Synthetic_vsamp", 
                       "Synthetic_lam", 
                       "Synthetic_kappa_a", 
                       "Synthetic_prior_r", 
                       "Synthetic_a_bs", 
                       "Synthetic_ndt", 
                       "Synthetic_v_max",  
                       "Synthetic_v_mod_stage_0", 
                       "Synthetic_v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 1, 1])


nump = len(p_names) # Parameter names
        
n_free_params = len(p_names)
        
param_sets = np.zeros((n_param_sets, n_free_params+1))

p_names.insert(0, "Synthetic_participant_ID")


df_param_sets = pd.DataFrame(param_sets, columns = p_names)
        
        
for i in range(n_param_sets):
    
    df_param_sets.loc[i,'Synthetic_participant_ID'] = i + 1
    
    for j in range(n_free_params):
        
        df_param_sets.loc[i,p_names[j+1]] = np.random.uniform(lower_bounds[j],upper_bounds[j])

## Save parameter sets

In [5]:
if model == "RL":
    directory = 'param_recovery_data/' + dataset + '/model_' + model 
    os.makedirs(directory, exist_ok=True)
    file_path = os.path.join(directory, "param_recovery_synthetic_params_M" + model + ".csv")
    
elif model == "RL_ddm" or model == "RL_ddm_biased":
    directory = 'param_recovery_data/' + dataset + '/model_' + model + "_DRM" + drmtype
    os.makedirs(directory, exist_ok=True)
    file_path = os.path.join(directory, "param_recovery_synthetic_params_M" + model + "_DRM" + drmtype + ".csv")

elif model == "AI":
    directory = 'param_recovery_data/' + dataset + '/model_' + model + str(mtype)
    os.makedirs(directory, exist_ok=True)
    file_path = os.path.join(directory, "param_recovery_synthetic_params_M" + model + str(mtype) + ".csv")

elif model == "AI_ddm":    
    directory = 'param_recovery_data/' + dataset + '/model_' + model + str(mtype) + "_DRM" + drmtype 
    os.makedirs(directory, exist_ok=True)
    file_path = os.path.join(directory, "param_recovery_synthetic_params_M" + model + str(mtype) +"_DRM" + drmtype + ".csv")
    
# Save the DataFrame to a CSV file
df_param_sets.to_csv(file_path, index=False)

## Simulate rounds for each parameter set (i.e., generate synthetic data)

In [6]:
list_dict_simulation_results = []

task_dict = {
    "type": "drift", # How outcome probabilities vary
    "x": False, # Whether transition probabilities are resampled
    "r": True, # Whether outcome probabilities are resampled
    "delta": 0.025, # The volatility of task statistics (variance of Gaussian for drift-version aka, diffusion rate)
    "bounds": [0.25,0.75] # List of 2 floats; lower and upper bounds of (final-stage) outcome probabilities
}

if dataset == 'magic_carpet_2020':
    task_dict['T'] = 202 

if dataset == 'magic_carpet_2023':
   task_dict['T'] = 150 
    

model_dict = {
    "act": model,
    "learn": learning,
    "drmtype": drmtype,
    "learn_transitions": True
}

for i in tqdm(range(n_param_sets)):

    # Get the parameter sets of this participant
    df_param_set = df_param_sets[df_param_sets['Synthetic_participant_ID'] == i+1]

    if model in ("AI_ddm","RL_ddm", "RL_ddm_biased"):

        model_dict['a_bs'] = df_param_set["Synthetic_a_bs"].iloc[0]
        model_dict['ndt'] = df_param_set["Synthetic_ndt"].iloc[0]

        if model == "AI_ddm":
        
            model_dict['lr'] = df_param_set["Synthetic_lr"].iloc[0]
            model_dict['lam'] = df_param_set["Synthetic_lam"].iloc[0]
            model_dict['kappa_a'] = df_param_set["Synthetic_kappa_a"].iloc[0] 
            model_dict['prior_r'] = df_param_set["Synthetic_prior_r"].iloc[0]
            
            if mtype == 0:
                model_dict['vunsamp'] = df_param_set["Synthetic_vunsamp"].iloc[0] 
                model_dict['vsamp'] = df_param_set["Synthetic_vsamp"].iloc[0] 
                model_dict['vps'] = df_param_set["Synthetic_vps"].iloc[0]
    
            elif mtype == 1:
                model_dict['vunsamp'] = df_param_set["Synthetic_vunsamp"].iloc[0]  
                model_dict['vsamp'] = 0
                model_dict['vps'] = df_param_set["Synthetic_vps"].iloc[0]
           
            elif mtype == 2:
                model_dict['vunsamp'] = 0 
                model_dict["vsamp"] = df_param_set["Synthetic_vsamp"].iloc[0]
                model_dict["vps"] = df_param_set["Synthetic_vps"].iloc[0]
    
            elif mtype == 3:
                model_dict["vunsamp"] = df_param_set["Synthetic_vunsamp"].iloc[0] 
                model_dict["vsamp"] = df_param_set["Synthetic_vsamp"].iloc[0]
                model_dict["vps"] = 0
        
        elif model in ("RL_ddm", "RL_ddm_biased"):
            model_dict['lr1'] = df_param_set['Synthetic_lr1'].iloc[0]
            model_dict['lr2'] = df_param_set['Synthetic_lr2'].iloc[0]
            model_dict['lam'] = df_param_set['Synthetic_lam'].iloc[0]
            model_dict['p'] = df_param_set['Synthetic_p'].iloc[0]
            model_dict['w'] = df_param_set['Synthetic_w'].iloc[0]
            
            if model == "RL_ddm_biased":
                model_dict['z_prime'] = df_param_set['Synthetic_z_prime'].iloc[0]
                         

        if drmtype == "linear":
            model_dict['v_stage_0'] = df_param_set["Synthetic_v_stage_0"].iloc[0]
            model_dict['v_stage_1'] = df_param_set["Synthetic_v_stage_1"].iloc[0]
            
        elif drmtype == "sigmoid":
            model_dict['v_mod_stage_0'] = df_param_set["Synthetic_v_mod_stage_0"].iloc[0]
            model_dict['v_mod_stage_1'] = df_param_set["Synthetic_v_mod_stage_1"].iloc[0]
            model_dict['v_max_stage_0'] = df_param_set["Synthetic_v_max_stage_0"].iloc[0]
            model_dict['v_max_stage_1'] = df_param_set["Synthetic_v_max_stage_1"].iloc[0]
        
        elif drmtype == "sigmoid_single_v_mod":
            model_dict['v_mod'] = df_param_set["Synthetic_v_mod"].iloc[0]
            model_dict['v_max_stage_0'] = df_param_set["Synthetic_v_max_stage_0"].iloc[0]
            model_dict['v_max_stage_1'] = df_param_set["Synthetic_v_max_stage_1"].iloc[0]
        
        elif drmtype == "sigmoid_single_v_max":
            model_dict['v_mod_stage_0'] = df_param_set["Synthetic_v_mod_stage_0"].iloc[0]
            model_dict['v_mod_stage_1'] = df_param_set["Synthetic_v_mod_stage_1"].iloc[0]
            model_dict['v_max'] = df_param_set["Synthetic_v_max"].iloc[0]

        agent = learn_and_act(task = task_dict, model = model_dict)
     
        actions, observations, rts, pi, p_trans, p_r, Gs = agent.perform_task()

        simulation_results_formatted = {"Synthetic_participant_ID": i+1,
                                        "choice1": actions[:,0].tolist(),
                                        "choice2": actions[:,1].tolist(),
                                        "rt1": rts[:,0].tolist(),
                                        "rt2": rts[:,1].tolist(),
                                        "final_state": observations[:,0].tolist(),
                                        "reward": observations[:,1].tolist()
                                       }

    else:
    
        if model == "RL":  
            model_dict['lr1'] = df_param_set['Synthetic_lr1'].iloc[0]
            model_dict['lr2'] = df_param_set['Synthetic_lr2'].iloc[0]
            model_dict['lam'] = df_param_set['Synthetic_lam'].iloc[0]
            model_dict['b1'] = df_param_set['Synthetic_b1'].iloc[0]
            model_dict['b2'] = df_param_set['Synthetic_b2'].iloc[0]
            model_dict['p'] = df_param_set['Synthetic_p'].iloc[0]
            model_dict['w'] = df_param_set['Synthetic_w'].iloc[0]
              
        elif model == "AI":
            model_dict['lr'] = df_param_set["Synthetic_lr"].iloc[0]
            model_dict['gamma1'] = df_param_set["Synthetic_gamma1"].iloc[0] 
            model_dict['gamma2'] = df_param_set["Synthetic_gamma2"].iloc[0] 
            model_dict['lam'] = df_param_set["Synthetic_lam"].iloc[0]
            model_dict['kappa_a'] = df_param_set["Synthetic_kappa_a"].iloc[0] 
            model_dict['prior_r'] = df_param_set["Synthetic_prior_r"].iloc[0]
            
            if mtype == 0:
                model_dict['vunsamp'] = df_param_set["Synthetic_vunsamp"].iloc[0] 
                model_dict['vsamp'] = df_param_set["Synthetic_vsamp"].iloc[0] 
                model_dict['vps'] = df_param_set["Synthetic_vps"].iloc[0]
    
            elif mtype == 1:
                model_dict["vunsamp"] = df_param_set["Synthetic_vunsamp"].iloc[0]
                model_dict['vsamp'] = 0 
                model_dict["vps"] = df_param_set["Synthetic_vps"].iloc[0]
                
       
            elif mtype == 2:
                model_dict["vunsamp"] = 0
                model_dict["vsamp"] = df_param_set["Synthetic_vsamp"].iloc[0]
                model_dict["vps"] = df_param_set["Synthetic_vps"].iloc[0]
    
            elif mtype == 3:
                model_dict["vunsamp"] = df_param_set["Synthetic_vunsamp"].iloc[0] 
                model_dict["vsamp"] = df_param_set["Synthetic_vsamp"].iloc[0]
                model_dict["vps"] = 0
                  
        agent = learn_and_act(task = task_dict, model = model_dict)
        
        actions, observations, pi, p_trans, p_r, Gs = agent.perform_task()
        
        
        simulation_results_formatted = {"Synthetic_participant_ID": i+1,
                                        "choice1": actions[:,0].tolist(),
                                        "choice2": actions[:,1].tolist(),
                                        "final_state": observations[:,0].tolist(),
                                        "reward": observations[:,1].tolist()
                                       }

    list_dict_simulation_results.append(simulation_results_formatted)

df_simulation_results = pd.DataFrame(list_dict_simulation_results)

100%|██████████| 23/23 [00:17<00:00,  1.29it/s]


## Save synthetic data

In [7]:
if model == "RL":
    file_path = os.path.join(directory, "param_recovery_synthetic_data_M" + model + ".csv")
    
elif model in ("RL_ddm", "RL_ddm_biased"):
    file_path = os.path.join(directory, "param_recovery_synthetic_data_M" + model + "_DRM" + drmtype + ".csv")

elif model == "AI":
    file_path = os.path.join(directory, "param_recovery_synthetic_data_M" + model + str(mtype) + ".csv")

elif model == "AI_ddm":    
    file_path = os.path.join(directory, "param_recovery_synthetic_data_M" + model + str(mtype) +"_DRM" + drmtype + ".csv")

df_simulation_results.to_csv(file_path, index=False)